## Sedentary and MVPA Outcomes Extraction
#### This notebook extracts 212 participants' sendentary and physical activity minutes from raw dataset with daily level granularity
#### The inputs and outputs are in csv format

In [1]:
import pandas as pd
import numpy as np
import pandasql as ps
import time 
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
#import tables
df_sed = pd.read_csv('Raw Data/sed_upload.csv')
df_pa = pd.read_csv('Raw Data/manual_min_upload.csv')
df_pa_device = pd.read_csv('Raw Data/s2_counts.csv')
df_user = pd.read_csv('Raw Data/users.csv') 

### Extract Participants (212 from intervention period)

In [3]:
#all participants
id_list = []
for i in range(df_user.shape[0]):
    #extract users' name starting with 'mbc2'
    if (df_user['username'][i][0:4] == 'mbc2'):
        id_list.append(df_user['user_id'][i])
id_list.sort()
print('There are', len(id_list), 'participants in intervention/follow-up period')

There are 212 participants in intervention/follow-up period


In [4]:
# selecting rows for participants only
df_sed= df_sed[df_sed['user_id'].isin(id_list)]
df_pa= df_pa[df_pa['user_id'].isin(id_list)]
df_pa_device= df_pa_device[df_pa_device['user_id'].isin(id_list)]

#reset index
df_sed = df_sed.reset_index(drop=True)
df_pa = df_pa.reset_index(drop=True)
df_pa_device = df_pa_device.reset_index(drop=True)

### SED (dayp1_minute + dayp2_minute + dayp3_minute +  dayp4_minute )

In [5]:
df_sed['sed_total'] = df_sed['dayp1_minute']+df_sed['dayp2_minute']+df_sed['dayp3_minute']+df_sed['dayp4_minute']

In [6]:
sed_list_included = ['user_id', 'when_sed', 'dayp1_minute','dayp2_minute', 'dayp3_minute', 'dayp4_minute', 'sed_total']
df_sed = df_sed[sed_list_included]

In [7]:
#cleaned sedentary outcome table
df_sed.head(5)

,user_id,when_sed,dayp1_minute,dayp2_minute,dayp3_minute,dayp4_minute,sed_total
0,14,2012-08-13,40,0,0,60,100
1,14,2012-08-14,60,0,30,0,90
2,9,2012-08-14,30,0,0,90,120
3,14,2012-08-15,0,60,0,180,240
4,9,2012-08-15,45,0,0,120,165


### PA (Manually entered time: min)

In [49]:
pa_list_included = ['user_id','tstamp_phone','min']
df_pa = df_pa[pa_list_included]

In [50]:
#cleaned self-raported MVPA outcome table
df_pa.head(5)

,user_id,tstamp_phone,min
0,14,2012-09-02 20:45:23,60
1,14,2012-09-02 20:46:01,90
2,1,2012-09-06 21:38:55,20
3,10000021,2012-09-07 11:21:24,30
4,1,2012-09-07 21:23:09,20


In [51]:
#get day-level date
df_pa['tstamp_phone'] = df_pa['tstamp_phone'].astype('string').str[:10]

#aggregate day-level minutes (sum)
df_pa= ps.sqldf("SELECT user_id, tstamp_phone as upload_time, sum(min) as pa_minute_app FROM df_pa group by user_id, tstamp_phone")

### MVPA (device)

In [10]:
df_pa_device.head(5)

,_id,user_id,uc_id,tstamp,pa_count,tstamp_server_uploaded,battery
0,1527,14,1,2012-08-13 06:52:19,158,2012-08-13 12:09:13,2663
1,1528,14,2,2012-08-13 06:53:19,52,2012-08-13 12:09:13,2663
2,1529,14,3,2012-08-13 06:54:19,52,2012-08-13 12:09:13,2663
3,1530,14,4,2012-08-13 06:55:19,52,2012-08-13 12:09:13,2663
4,1531,14,5,2012-08-13 06:56:19,53,2012-08-13 12:09:13,2663


In [11]:
#get day-level date
df_pa_device['upload_time'] = df_pa_device['tstamp'].astype('string').str[:10]

#### Shimmer accelerometer  device.   pa_count > 1900 is activity minute ,  65 < pa_count  < 1900 is wear minute 

In [12]:
df_pa_device_active = df_pa_device[df_pa_device['pa_count'] >= 1900]
df_pa_device_active

,_id,user_id,uc_id,tstamp,pa_count,tstamp_server_uploaded,battery,upload_time
2254,3841,14,1526,2012-08-14 23:52:47,1923,2012-08-15 04:32:02,2433,2012-08-14
2584,4191,14,1836,2012-08-15 10:17:29,2164,2012-08-15 10:18:59,2551,2012-08-15
7647,9254,14,4483,2012-08-19 08:25:51,2148,2012-08-19 08:28:50,2631,2012-08-19
7648,9255,14,4484,2012-08-19 08:26:51,2192,2012-08-19 08:28:50,2631,2012-08-19
7649,9256,14,4485,2012-08-19 08:27:51,2047,2012-08-19 08:32:58,2631,2012-08-19
...,...,...,...,...,...,...,...,...
12962686,13047728,583,5664,2015-03-13 18:05:36,2645,2015-03-13 18:12:55,2619,2015-03-13
12962687,13047729,583,5665,2015-03-13 18:06:39,2662,2015-03-13 18:15:06,2619,2015-03-13
12962692,13047734,583,5670,2015-03-13 18:11:41,2788,2015-03-13 18:19:22,2609,2015-03-13
12962693,13047735,583,5671,2015-03-13 18:12:41,2602,2015-03-13 18:23:36,2609,2015-03-13


In [13]:
#group by user and each day, count total physical activity minutes per day
df_pa_device_count = ps.sqldf("select user_id, upload_time, count(pa_count) as pa_minute_shimmer from df_pa_device_active group by user_id, upload_time")

In [14]:
#cleaned device-derived MVPA outcome table
df_pa_device_count.head(5)

,user_id,upload_time,pa_minute_shimmer
0,1,2012-08-27,2
1,1,2012-09-07,2
2,1,2012-09-08,19
3,1,2012-09-11,5
4,1,2012-09-16,4


In [31]:
#self-reported
df_pa.head(5)

,user_id,upload_time,total_min
0,1,2012-09-06,20
1,1,2012-09-07,20
2,1,2012-09-10,20
3,1,2012-09-12,20
4,1,2012-09-16,20


### Combine self-reported and Shimmer MVPA 

In [52]:
combine = pd.merge(df_pa, df_pa_device_count, how='outer', on=['user_id', 'upload_time'])

In [53]:
combine.head(5)

,user_id,upload_time,pa_minute_app,pa_minute_shimmer
0,1,2012-09-06,20.0,NaN
1,1,2012-09-07,20.0,2.0
2,1,2012-09-10,20.0,NaN
3,1,2012-09-12,20.0,NaN
4,1,2012-09-16,20.0,4.0


In [54]:
#fill missing values with 0
combine['pa_minute_app'] = combine['pa_minute_app'].fillna(0)
combine['pa_minute_shimmer'] = combine['pa_minute_shimmer'].fillna(0)

#get combined outcome (sum)
combine['MVPA_min'] = combine['pa_minute_app'] + combine['pa_minute_shimmer']

In [55]:
combine.head(5)

,user_id,upload_time,pa_minute_app,pa_minute_shimmer,MVPA_min
0,1,2012-09-06,20.0,0.0,20.0
1,1,2012-09-07,20.0,2.0,22.0
2,1,2012-09-10,20.0,0.0,20.0
3,1,2012-09-12,20.0,0.0,20.0
4,1,2012-09-16,20.0,4.0,24.0


### Save the results (local csv)

In [79]:
df_sed.to_csv('Result/sed_clean.csv', index=False)

In [80]:
df_pa.to_csv('Result/pa_clean.csv', index=False)

In [81]:
df_pa_device_count.to_csv('Result/pa_device_clean.csv', index=False)

In [57]:
combine.to_csv('Result/pa_merged.csv', index=False)